In [9]:
!pip install -q \
    pyannote.audio \
    faiss-cpu \
    sentence-transformers \
    openai \
    langchain_experimental \
    kiwipiepy \
    torchaudio \
    torch \
    numpy \
    pandas \
    tqdm \
    huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 62.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
pip install pydub

In [11]:
!pip install pymysql

In [12]:
pip install -U langchain langchain_openai

In [13]:
!pip install fastapi uvicorn

In [14]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-oyy9tz3l
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-oyy9tz3l
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import re
import os
import torch
import faiss
import numpy as np
import openai
from pyannote.audio import Pipeline
import whisper
from sentence_transformers import SentenceTransformer
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings
from collections import Counter
from kiwipiepy import Kiwi
from fastapi import FastAPI
from pydantic import BaseModel
from pydub import AudioSegment
# send pipeline to GPU (when available)
import torch
import pymysql
import json
from pydub import AudioSegment


# ✅ FastAPI 서버 생성
app = FastAPI()

# ✅ MySQL 연결 설정
DB_CONFIG = {
    "host": "project-db-cgi.smhrd.com",
    "port": 3307,
    "user": "TTEAM",
    "password": "11111",
    "database": "TTEAM"
}

# 🔹 Hugging Face & OpenAI API 키 설정
HUGGINGFACE_TOKEN = ""
os.environ["OPENAI_API_KEY"] = ""

# ✅ GPU 사용 여부 확인
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 실행 기기: {DEVICE}")

# ✅ Pydantic 데이터 모델
class AudioProcessingRequest(BaseModel):
    meeting_idx: int #36
    user_idx: int #3
    file_name: str #/content/drive/MyDrive/실전/record/aaa.wav  # 업로드된 파일 URL

# ✅ OpenAI 클라이언트 생성
client = openai.OpenAI()

# ✅ Faiss DB 저장 경로
FAISS_DIR = "/content/drive/MyDrive/실전/faiss"
FAISS_INDEX_FILE = os.path.join(FAISS_DIR, "faiss_test.index")
# os.makedirs(FAISS_DIR, exist_ok=True)

# ✅ Faiss DB 로드 또는 초기화
def load_or_init_faiss(embedding_dimension):
    if os.path.exists(FAISS_INDEX_FILE):
        return faiss.read_index(FAISS_INDEX_FILE)
    else:
        base_index = faiss.IndexFlatL2(embedding_dimension)
        return faiss.IndexIDMap2(base_index)

# ✅ Faiss에서 현재 저장된 마지막 `chunk_idx` 가져오기
def get_next_chunk_idx_faiss(index):
    if index.ntotal == 0:
        return 1  # 첫 번째 청크라면 1부터 시작

    stored_ids = faiss.vector_to_array(index.id_map)
    last_chunk_idx = np.max(stored_ids)

    return int(last_chunk_idx) + 1  # 마지막 `chunk_idx`에서 +1 증가

# ✅ 청크 분할 및 MySQL 저장 함수
def save_chunks_to_db(chunks, meeting_idx, user_idx, embeddings):
    conn = pymysql.connect(**DB_CONFIG)
    try:
        with conn.cursor() as cursor:
            sql_chunks = """
            INSERT INTO chunks (chunk_idx, meeting_idx, user_idx, chunk_text, created_at)
            VALUES (%s, %s, %s, %s, NOW())
            """

            index = load_or_init_faiss(embeddings.shape[1])
            next_chunk_idx = get_next_chunk_idx_faiss(index)
            print(next_chunk_idx)

            ids = []
            vectors = []

            for i, chunk in enumerate(chunks):
                current_idx = next_chunk_idx + i
                cursor.execute(sql_chunks, (current_idx, meeting_idx, user_idx, chunk))

                ids.append(current_idx)
                vectors.append(embeddings[i])

            index.add_with_ids(np.array(vectors), np.array(ids))
            faiss.write_index(index, FAISS_INDEX_FILE)

        conn.commit()
    finally:
        conn.close()

# 🔹 시간을 `MM:SS` 형식으로 변환
def format_time(seconds):
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02}:{seconds:02}"

# 🔹 같은 참여자의 연속된 발언을 병합
def merge_speaker_statements(speaker_texts):
    merged_results = []
    previous_speaker = None
    previous_text = ""
    previous_time = ""

    for line in speaker_texts:
        match = re.match(r"(참여자_\d+) (\d{2}:\d{2}) (.+)", line)
        if match:
            speaker, time, text = match.groups()
            if speaker == previous_speaker:
                previous_text += " " + text
            else:
                if previous_speaker is not None:
                    merged_results.append(f"{previous_speaker} {previous_time} {previous_text}")
                previous_speaker, previous_text, previous_time = speaker, text, time

    if previous_speaker is not None:
        merged_results.append(f"{previous_speaker} {previous_time} {previous_text}")

    return merged_results




# 🔹 Faiss DB 로드
def load_or_init_faiss(embedding_dimension):
    if os.path.exists(FAISS_INDEX_FILE):
        return faiss.read_index(FAISS_INDEX_FILE)
    else:
        base_index = faiss.IndexFlatL2(embedding_dimension)
        return faiss.IndexIDMap2(base_index)


# 🔹 Faiss DB 저장
def save_faiss(index):
    faiss.write_index(index, FAISS_INDEX_FILE)



# # 🔹 Faiss 검색 테스트
# query_text = "WTO 의료시장 개방"
# query_embedding = kosimcse_model.encode([query_text])
# distances, indices = index.search(np.array(query_embedding), 3)

# print("\n🔍 검색 결과:")
# for rank, (idx, distance) in enumerate(zip(indices[0], distances[0]), start=1):
#     print(f"  {rank}. ID: {idx}, 거리: {distance}")

# 🔹 요약 생성
def generate_summary(text_with_speakers):
    prompts = {
        "summary": "다음 회의록을 요약해줘.",
        "subject": "다음 회의록에서 주요 주제를 한 문장으로 요약해줘.",
        "positive": "다음 회의록을 기반으로 긍정적인 피드백을 작성해줘.참여자들에게도",
        "negative": "다음 회의록을 기반으로 부정적인 피드백을 작성해줘.참여자들에게도"
    }

    summary_json = {}  # 결과 저장용 딕셔너리

    for key, prompt in prompts.items():
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # 최신 모델 사용
                messages=[
                    {"role": "system", "content": "당신은 전문 회의 요약가입니다."},
                    {"role": "user", "content": f"{prompt}\n\n{text_with_speakers}"}
                ],
                max_tokens=500,  # 요약 길이 제한
                temperature=0.3,  # 안정적인 답변을 위해 낮은 온도 설정
            )
            summary_json[key] = response.choices[0].message.content.strip()
        except Exception as e:
            summary_json[key] = f"⚠️ 요약 생성 실패: {str(e)}"

    return summary_json

# ✅ 음성 파일의 총 길이 추출
def get_audio_length(file_path):
    audio = AudioSegment.from_file(file_path)
    return int(len(audio) / 1000)  # ms → 초 변환

# ✅ Kiwi 기반 키워드 추출 함수 (전체 키워드 가져오기)
def extract_keywords(text):
    kiwi = Kiwi()
    tokens = kiwi.tokenize(text)

    # 🔹 명사(NNG, NNP)만 추출
    nouns = [token.form for token in tokens if token.tag in ['NNG', 'NNP']]

    # 🔹 등장 횟수 계산
    word_counts = Counter(nouns)

    # 🔹 JSON 형태로 반환 (키워드: 등장 횟수)
    return dict(word_counts)


# ✅ FastAPI 엔드포인트: 음성 처리 및 MySQL 저장
@app.post("/process_audio/")
async def process_audio(request: AudioProcessingRequest):
    file_path = os.path.join("/content/drive/MyDrive/실전/data/records", request.file_name)

    # ✅ 음성 파일 길이 가져오기
    record_time = get_audio_length(file_path)


    # 🔹 Pyannote 화자 분리 모델 로드
    print("🔍 화자 분리 모델 다운로드 중...")
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=HUGGINGFACE_TOKEN)

    # ✅ 화자 분리 실행
    diarization = pipeline(file_path)
    pipeline.to(torch.device("cuda"))


    # ✅ STT 모델 선언

    model = whisper.load_model("large-v3", device=DEVICE)


    # ✅ STT 변환 실행
    speaker_map = {}
    speaker_count = 1
    speaker_texts = []

    audio = AudioSegment.from_wav(file_path)
    print(file_path)
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start_ms = int(turn.start * 1000)  # 초를 밀리초로 변환
        end_ms = int(turn.end * 1000)
        start_time_formatted = format_time(turn.start)  # ⏳ 시간 변환 (MM:SS)
        print(start_time_formatted)
        # 오디오 구간 추출
        segment = audio[start_ms:end_ms]
        segment_file = f"segment_{speaker}_{start_ms}.wav"
        segment.export(segment_file, format="wav")

        # Whisper로 STT 수행
        result = model.transcribe(segment_file, language='ko')
        text = result["text"]
        print(text)
        speaker = str(speaker).replace('SPEAKER', '참여자')
        speaker_texts.append(f'{speaker} {start_time_formatted} {text}')


    # 🔹 같은 참여자의 발언 병합
    merged_texts = merge_speaker_statements(speaker_texts)
    print(merged_texts)

    cleaned_text = "\n".join(merged_texts)
    print(f"✅ STT 완료! 텍스트 길이: {len(cleaned_text)}")

    # 🔹 키워드 & 청크 분할용 화자 정보 제거
    text_without_speakers = re.sub(r"참여자_\d+ \d{2}:\d{2} ", "", cleaned_text)

    # 🔹 청크 분할
    print("📝 텍스트 청크 분할 중...")
    text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type="percentile", breakpoint_threshold_amount=70)
    chunks = text_splitter.split_text(text_without_speakers)
    print(f"✅ 청크 분할 완료! 총 {len(chunks)}개")

    # 🔹 KoSimCSE로 임베딩
    print("📊 KoSimCSE Embedding 생성 중...")
    kosimcse_model = SentenceTransformer("BM-K/KoSimCSE-roberta-multitask", device=DEVICE)
    embeddings = kosimcse_model.encode(chunks, convert_to_tensor=True).cpu().numpy()
    embedding_dimension = embeddings.shape[1]
    print(f"✅ 임베딩 완료! 벡터 크기: {embedding_dimension}")

    index = load_or_init_faiss(embedding_dimension)

    # ✅ 키워드 추출
    keywords = extract_keywords(text_without_speakers)

    # ✅ 요약 생성
    summary_json = generate_summary(cleaned_text)
    print(f"✅ 요약 완료! \n{summary_json}")

    # ✅ MySQL 저장
    conn = pymysql.connect(**DB_CONFIG)
    try:
        with conn.cursor() as cursor:
            # 🔹 records 테이블에 데이터 저장

            sql_records = """
            INSERT INTO records (meeting_idx, file_url, record_time, record_text, text_summary, uploaded_at, text_keyword)
            VALUES (%s, %s, %s, %s, %s, now(), %s)
            """
            print(request.meeting_idx)

            cursor.execute(sql_records, (
                request.meeting_idx, request.file_name, record_time, cleaned_text,
                json.dumps(summary_json, ensure_ascii=False),
                json.dumps(keywords, ensure_ascii=False)
            ))
            print(request.meeting_idx)


        conn.commit()
    finally:
        conn.close()

    # ✅ Faiss 및 MySQL에 저장
    save_chunks_to_db(chunks, request.meeting_idx, request.user_idx, embeddings)

    return {
        "status": "success",
        "meeting_idx": request.meeting_idx,
        "record_time": record_time,
        "record_text": cleaned_text,
        "text_summary": summary_json,
        "text_keyword": keywords,
        "chunks_count": len(chunks)
    }

🚀 실행 기기: cuda


In [16]:
# 필요한 라이브러리 설치
!pip install pyngrok

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.set_auth_token("2ubn5t4R3wCYRMwoqiNdutdH09j_4qNznUTidUZPKuvCBpBAx")
ngrok_tunnel = ngrok.connect(8001)
print('공용 URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8001)

INFO:     Started server process [52208]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


공용 URL: https://104f-34-143-154-113.ngrok-free.app
